<a href="https://colab.research.google.com/github/DanielP0111/REU-Code/blob/main/eatdcorupus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# working with chinese dataset

# upload zip file of chinese audio
from google.colab import files
uploaded = files.upload()

Saving EATD-Corpus.zip to EATD-Corpus.zip


In [ ]:
# unzips
import zipfile
import os

with zipfile.ZipFile("EATD-Corpus.zip", 'r') as zip_ref:
    zip_ref.extractall("EATD-Corpus")

In [ ]:
import os
print(os.listdir("EATD-Corpus"))


['__MACOSX', 'EATD-Corpus']


In [ ]:
# loads data from zip
import librosa
import numpy as np
import os
import glob

def extract_mel_spectrogram(audio_path):
    y, sr = librosa.load(audio_path, sr=None)
    mel = librosa.feature.melspectrogram(y=y, sr=sr)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    return mel_db

X = []
y = []

base_path = "EATD-Corpus/EATD-Corpus"

for participant_folder in os.listdir(base_path):
    participant_path = os.path.join(base_path, participant_folder)

    if os.path.isdir(participant_path):
        label_path = os.path.join(participant_path, "new_label.txt")
        if not os.path.exists(label_path):
            continue

        with open(label_path, 'r') as f:
            try:
                label_score = float(f.read().strip())
            except ValueError:
                continue

        label = 1 if label_score >= 53 else 0

        # use all three files
        mel_features = []
        for mood in ["positive_out.wav", "negative_out.wav", "neutral_out.wav"]:
            wav_path = os.path.join(participant_path, mood)
            if os.path.exists(wav_path):
                mel = extract_mel_spectrogram(wav_path)
                mel = librosa.util.fix_length(mel, size=128, axis=1)
                mel_features.append(mel)

        if mel_features:
            combined = np.mean(np.stack(mel_features), axis=0)
            X.append(combined)
            y.append(label)

X = np.array(X)
y = np.array(y)




/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


In [ ]:
# logistic model for only the real audio
# 0 = healthy
# 1 = depressed

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report

X_flat = X.reshape(X.shape[0], -1)

X_train, X_test, y_train, y_test = train_test_split(X_flat, y, test_size=0.2, random_state=42)

# train
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("\nClassification Report:\n", report)


Accuracy: 0.8181818181818182
F1 Score: 0.0

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.96      0.90        28
           1       0.00      0.00      0.00         5

    accuracy                           0.82        33
   macro avg       0.42      0.48      0.45        33
weighted avg       0.72      0.82      0.76        33



In [ ]:
!pip install --upgrade google-cloud-texttospeech

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 kB 2.7 MB/s eta 0:00:00


In [ ]:
# start of creating synthetic data

from google.colab import files
uploaded = files.upload()

Saving gen-lang-client-0491551571-7ed0b4194128.json to gen-lang-client-0491551571-7ed0b4194128.json


In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "gen-lang-client-0491551571-7ed0b4194128.json"

In [ ]:
from google.cloud import texttospeech

client = texttospeech.TextToSpeechClient()

def synthesize_chinese(text, out_file):
    input_text = texttospeech.SynthesisInput(text=text)

    # Mandarin
    voice = texttospeech.VoiceSelectionParams(
        language_code="cmn-CN",
        ssml_gender=texttospeech.SsmlVoiceGender.FEMALE
    )
# WAV format
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.LINEAR16
    )

    response = client.synthesize_speech(
        input=input_text, voice=voice, audio_config=audio_config
    )

    with open(out_file, "wb") as out:
        out.write(response.audio_content)
    print(f"Saved: {out_file}")


In [ ]:
# upload EATD-Corpus.zip
from google.colab import files
uploaded = files.upload()

Saving EATD-Corpus.zip to EATD-Corpus.zip


In [ ]:
import zipfile

with zipfile.ZipFile("EATD-Corpus.zip", 'r') as zip_ref:
    zip_ref.extractall("EATD-Corpus")

In [ ]:
import os
print(os.listdir("EATD-Corpus"))

['__MACOSX', 'EATD-Corpus']


In [ ]:

import os
import csv

base_path = "EATD-Corpus"
if "EATD-Corpus" in os.listdir(base_path):
    base_path = os.path.join(base_path, "EATD-Corpus")

csv_filename = "EATD_transcripts.csv"
with open(csv_filename, mode="w", encoding="utf-8-sig", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["participant_id", "mood", "transcript"])

    for folder_name in sorted(os.listdir(base_path)):
        folder_path = os.path.join(base_path, folder_name)
        if not os.path.isdir(folder_path):
            continue
        if not (folder_name.startswith("t_") or folder_name.startswith("v_")):
            continue

        for mood in ["positive", "neutral", "negative"]:
            file_path = os.path.join(folder_path, f"{mood}.txt")
            if os.path.exists(file_path):
                with open(file_path, "r", encoding="utf-8") as f:
                    text = f.read().strip()
                writer.writerow([folder_name, mood, text])
            else:
                print(f"Missing file: {file_path}")

print(f"All transcripts saved to: {csv_filename}")


All transcripts saved to: EATD_transcripts.csv


In [ ]:
# made .csv transcript of all chinese data
from google.colab import files
files.download("EATD_transcripts.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# upload json file
from google.colab import files
uploaded = files.upload()

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'gen-lang-client-0491551571-7ed0b4194128.json'


Saving gen-lang-client-0491551571-7ed0b4194128.json to gen-lang-client-0491551571-7ed0b4194128 (1).json


In [ ]:
# tester created 20 chinese samples with only female

import pandas as pd
import os
from sklearn.model_selection import train_test_split
from google.cloud import texttospeech

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'gen-lang-client-0491551571-7ed0b4194128.json'

df = pd.read_csv("EATD_transcripts.csv", encoding="utf-8-sig")
assert 'transcript' in df.columns, "CSV must contain a column named 'transcript'"

df = df[df['transcript'].str.strip().str.len() > 3]

# split 80/20
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df = train_df.reset_index(drop=True)

client = texttospeech.TextToSpeechClient()

voice = texttospeech.VoiceSelectionParams(
    language_code="cmn-CN",
    name="cmn-CN-Wavenet-A",
    ssml_gender=texttospeech.SsmlVoiceGender.FEMALE
)

audio_config = texttospeech.AudioConfig(
    audio_encoding=texttospeech.AudioEncoding.LINEAR16
)

os.makedirs("synthetic_audio", exist_ok=True)

output_files = []
processed_rows = []

for idx, row in train_df.head(30).iterrows():
    try:
        text = row['transcript']
        filename = f"synthetic_audio/train_sample_{idx}.wav"

        synthesis_input = texttospeech.SynthesisInput(text=text)

        response = client.synthesize_speech(
            input=synthesis_input,
            voice=voice,
            audio_config=audio_config
        )

        with open(filename, "wb") as out:
            out.write(response.audio_content)

        output_files.append(filename)
        processed_rows.append(row)

        print(f"Generated: {filename}")

    except Exception as e:
        print(f"Error at row {idx}: {e}")

train_subset = pd.DataFrame(processed_rows)
train_subset['audio_file'] = output_files
train_subset.to_csv("EATD_train_with_audio.csv", index=False, encoding="utf-8-sig")
print("📁 Saved mapping CSV: EATD_train_with_audio.csv")


Generated: synthetic_audio/train_sample_0.wav
Generated: synthetic_audio/train_sample_1.wav
Generated: synthetic_audio/train_sample_2.wav
Generated: synthetic_audio/train_sample_3.wav
Generated: synthetic_audio/train_sample_4.wav
Generated: synthetic_audio/train_sample_5.wav
Generated: synthetic_audio/train_sample_6.wav
Generated: synthetic_audio/train_sample_7.wav
Generated: synthetic_audio/train_sample_8.wav
Generated: synthetic_audio/train_sample_9.wav
Generated: synthetic_audio/train_sample_10.wav
Generated: synthetic_audio/train_sample_11.wav
Generated: synthetic_audio/train_sample_12.wav
Generated: synthetic_audio/train_sample_13.wav
Generated: synthetic_audio/train_sample_14.wav
Generated: synthetic_audio/train_sample_15.wav
Generated: synthetic_audio/train_sample_16.wav
Generated: synthetic_audio/train_sample_17.wav
Generated: synthetic_audio/train_sample_18.wav
Generated: synthetic_audio/train_sample_19.wav
Generated: synthetic_audio/train_sample_20.wav
Generated: synthetic_au

In [ ]:
# tester created 20 chinese samples with only female
import shutil

shutil.make_archive("synthetic_audio", 'zip', "synthetic_audio")


'/content/synthetic_audio.zip'

In [ ]:
from google.colab import files

files.download("synthetic_audio.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# all 486 synthetic audio files
import pandas as pd
import os
from google.cloud import texttospeech
from sklearn.utils import shuffle

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'gen-lang-client-0491551571-7ed0b4194128.json'

df = pd.read_csv("EATD_transcripts.csv", encoding="utf-8-sig")
df = df[df['transcript'].str.strip().str.len() > 3]
df = shuffle(df, random_state=42).reset_index(drop=True)

# split into half male, half female
half = len(df) // 2
df_female = df.iloc[:half].copy()
df_male = df.iloc[half:].copy()

client = texttospeech.TextToSpeechClient()

voice_female = texttospeech.VoiceSelectionParams(
    language_code="cmn-CN",
    name="cmn-CN-Wavenet-A",
    ssml_gender=texttospeech.SsmlVoiceGender.FEMALE
)

voice_male = texttospeech.VoiceSelectionParams(
    language_code="cmn-CN",
    name="cmn-CN-Wavenet-B",
    ssml_gender=texttospeech.SsmlVoiceGender.MALE
)

audio_config = texttospeech.AudioConfig(
    audio_encoding=texttospeech.AudioEncoding.LINEAR16
)

os.makedirs("synthetic_audio_all", exist_ok=True)

def synthesize_and_save(row, idx, voice):
    text = row['transcript']
    filename = f"synthetic_audio_all/sample_{idx}.wav"
    synthesis_input = texttospeech.SynthesisInput(text=text)

    try:
        response = client.synthesize_speech(
            input=synthesis_input,
            voice=voice,
            audio_config=audio_config
        )
        with open(filename, "wb") as out:
            out.write(response.audio_content)
        return filename
    except Exception as e:
        print(f"Error at row {idx}: {e}")
        return None

output_files = []
genders = []
transcripts = []
participants = []
moods = []

# female voices
for idx, row in df_female.iterrows():
    path = synthesize_and_save(row, idx, voice_female)
    if path:
        output_files.append(path)
        genders.append("female")
        transcripts.append(row['transcript'])
        participants.append(row['participant_id'])
        moods.append(row['mood'])

# male voices
for idx, row in df_male.iterrows():
    path = synthesize_and_save(row, idx + half, voice_male)
    if path:
        output_files.append(path)
        genders.append("male")
        transcripts.append(row['transcript'])
        participants.append(row['participant_id'])
        moods.append(row['mood'])

df_output = pd.DataFrame({
    "participant_id": participants,
    "mood": moods,
    "gender": genders,
    "transcript": transcripts,
    "audio_file": output_files
})

df_output.to_csv("EATD_all_with_gender_audio.csv", index=False, encoding="utf-8-sig")
print("Saved full audio metadata: EATD_all_with_gender_audio.csv")


Saved full audio metadata: EATD_all_with_gender_audio.csv


In [ ]:
# all 486 synthetic audio files
import shutil
from google.colab import files

shutil.make_archive("synthetic_audio_all", 'zip', "synthetic_audio_all")
files.download("synthetic_audio_all.zip")

files.download("EATD_all_with_gender_audio.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from collections import Counter

def extract_features(audio_path):
    try:
        y_audio, sr = librosa.load(audio_path, sr=None)
        mel_spec = librosa.feature.melspectrogram(y=y_audio, sr=sr, n_mels=128)
        log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
        feature = np.concatenate([np.mean(log_mel_spec, axis=1), np.std(log_mel_spec, axis=1)])
        return feature
    except Exception as e:
        print(f"Error extracting {audio_path}: {e}")
        return None

real_data_path = "/mnt/data/EATD-Corpus"
synthetic_data_path = "/mnt/data/chinese_synthetic_audio"
X_real = []
y_real = []

labels_csv_path = "/mnt/data/EATD_labels.csv"

labels_df = pd.read_csv(labels_csv_path)

label_map = dict(zip(labels_df['participant_id'], labels_df['label']))

print(f"Loaded {len(label_map)} participant labels.")

for participant_id in os.listdir(real_data_path):
    participant_folder = os.path.join(real_data_path, participant_id)
    if os.path.isdir(participant_folder):
        label = label_map.get(participant_id, None)
        if label is None:
            continue
        for wav_file in os.listdir(participant_folder):
            if wav_file.endswith(".wav"):
                audio_file_path = os.path.join(participant_folder, wav_file)
                feat = extract_features(audio_file_path)
                if feat is not None:
                    X_real.append(feat)
                    y_real.append(label)

print(f"Loaded {len(X_real)} real audio samples.")

X_syn = []
y_syn = []

for fname in os.listdir(synthetic_data_path):
    if fname.endswith(".wav"):
        path = os.path.join(synthetic_data_path, fname)
        feat = extract_features(path)
        if feat is not None:
            X_syn.append(feat)
            y_syn.append(1)

print(f"Loaded {len(X_syn)} synthetic audio samples.")

X_real_train, X_real_test, y_real_train, y_real_test = train_test_split(
    X_real, y_real, test_size=0.2, random_state=42, stratify=y_real
)

print(f"Real train: {len(X_real_train)} samples")
print(f"Real test: {len(X_real_test)} samples")

X_train = np.array(X_real_train + X_syn)
y_train = np.array(y_real_train + y_syn)

X_test = np.array(X_real_test)
y_test = np.array(y_real_test)

print("\nSample counts:")
print("Training set labels:", Counter(y_train))
print("Test set labels:", Counter(y_test))

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("\nEvaluation results:")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Classification Report:\n{report}")


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/EATD_labels.csv'